In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itbagdt" #TODO
fs_method, fs_ratio = "svc", 0.5  #dnp & graces
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9180 | AUPR: 0.6514 | Acc: 95.30%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9225 | AUPR: 0.6639 | Acc: 95.33%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9257 | AUPR: 0.6773 | Acc: 95.47%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9208 | AUPR: 0.6587 | Acc: 95.38%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9246 | AUPR: 0.6724 | Acc: 95.44%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9257 | AUPR: 0.6795 | Acc: 95.43%
best model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9257 | AUPR: 0.6795 | Acc: 95.43%
Internal test: AUC: 0.9324 | AUPR: 0.6977 | Acc: 95.28%
External test: AUC: 0.9187 | AUPR: 0.6656 | Acc: 95.11%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9202 | AUPR: 0.6267 | Acc: 95.10%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9215 | AUPR: 0.6247 | Acc: 95.00%
model(kernel=linear, C=1)
Valid:         AUC: 0.9210 | AUPR: 0.6279 | Acc: 95.07%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9215 | AUPR: 0.6247 | Acc: 95.00%
Internal test: AUC: 0.9290 | AUPR: 0.6708 | Acc: 94.84%
External test: AUC: 0.9158 | AUPR: 0.6065 | Acc: 94.57%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7695 | AUPR: 0.4457 | Acc: 94.86%
model(n_neighbors=5)
Valid:         AUC: 0.7932 | AUPR: 0.4962 | Acc: 95.16%
model(n_neighbors=7)
Valid:         AUC: 0.8107 | AUPR: 0.5250 | Acc: 95.29%
best model(n_neighbors=7)
Valid:         AUC: 0.8107 | AUPR: 0.5250 | Acc: 95.29%
Internal test: AUC: 0.8376 | AUPR: 0.5808 | Acc: 95.04%
External test: AUC: 0.8180 | AUPR: 0.5371 | Acc: 95.05%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8850 | AUPR: 0.4551 | Acc: 86.68%
best model()
Valid:         AUC: 0.8850 | AUPR: 0.4551 | Acc: 86.68%
Internal test: AUC: 0.8976 | AUPR: 0.4890 | Acc: 86.13%
External test: AUC: 0.8856 | AUPR: 0.4410 | Acc: 84.68%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression


model(C=1.0)
Valid:         AUC: 0.9222 | AUPR: 0.6517 | Acc: 95.44%
model(C=10.0)
Valid:         AUC: 0.9219 | AUPR: 0.6510 | Acc: 95.46%
model(C=100.0)
Valid:         AUC: 0.9215 | AUPR: 0.6505 | Acc: 95.46%
best model(C=1.0)
Valid:         AUC: 0.9222 | AUPR: 0.6517 | Acc: 95.44%
Internal test: AUC: 0.9292 | AUPR: 0.6898 | Acc: 94.99%
External test: AUC: 0.9149 | AUPR: 0.6337 | Acc: 94.92%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8630 | AUPR: 0.5527 | Acc: 86.93%
model(criterion=log_loss)
Valid:         AUC: 0.8691 | AUPR: 0.5601 | Acc: 86.90%
model(criterion=entropy)
Valid:         AUC: 0.8691 | AUPR: 0.5601 | Acc: 86.90%
best model(criterion=log_loss)
Valid:         AUC: 0.8691 | AUPR: 0.5601 | Acc: 86.90%
Internal test: AUC: 0.8710 | AUPR: 0.6068 | Acc: 83.72%
External test: AUC: 0.8670 | AUPR: 0.5521 | Acc: 83.19%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9139 | AUPR: 0.6521 | Acc: 90.27%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9153 | AUPR: 0.6557 | Acc: 90.12%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9152 | AUPR: 0.6557 | Acc: 90.00%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9130 | AUPR: 0.6560 | Acc: 90.27%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9153 | AUPR: 0.6584 | Acc: 90.31%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9153 | AUPR: 0.6593 | Acc: 90.24%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9130 | AUPR: 0.6560 | Acc: 90.27%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9153 | AUPR: 0.6584 | Acc: 90.31%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9153 | AUPR: 0.6593 | Acc: 90.24%
best model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9153 | AUPR: 0.6557 | Acc: 9

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9193 | AUPR: 0.6461 | Acc: 95.27%
model(criterion=squared_error)
Valid:         AUC: 0.9186 | AUPR: 0.6471 | Acc: 95.26%
best model(criterion=friedman_mse)
Valid:         AUC: 0.9193 | AUPR: 0.6461 | Acc: 95.27%
Internal test: AUC: 0.9268 | AUPR: 0.6917 | Acc: 95.18%
External test: AUC: 0.9180 | AUPR: 0.6485 | Acc: 95.02%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9163 | AUPR: 0.6356 | Acc: 95.10%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9170 | AUPR: 0.6354 | Acc: 95.13%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9177 | AUPR: 0.6360 | Acc: 95.19%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9177 | AUPR: 0.6360 | Acc: 95.19%
Internal test: AUC: 0.9216 | AUPR: 0.6748 | Acc: 94.94%
External test: AUC: 0.9122 | AUPR: 0.6172 | Acc: 94.63%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)